In [1]:
import rospy
from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach
import os, shutil

from ipywidgets import FloatProgress
from IPython.display import display
import time

In [2]:
vc = VirtualCoach(environment='local', storage_username='nrpuser')
# Type "password"

INFO: [2018-09-26 12:23:41,933 - Configuration] Loading configuration file config.json
INFO: [2018-09-26 12:23:41,936 - Configuration] Using user specified environment: local
INFO: [2018-09-26 12:23:41,941 - VirtualCoach] Logging into the Storage Server as: nrpuser
········
INFO: [2018-09-26 12:23:46,633 - VirtualCoach] Ready.


In [3]:
configName = "fable_cerebellum"

In [391]:
folder = "/home/dtu-neurorobotics/Documents/workspace/Projects/DTU-Pavia_Cerebellum_Carlos/cereb_tuning/results/"
test = "54"
folder = folder+test
try:
    file_list = os.listdir(folder)
except Exception as e:
    print("Creating experiment folder: " + test)
    os.mkdir(folder)
    file_list = os.listdir(folder)
    
    
        
if len(file_list):
    print("Going to delete everything here:")
    print(file_list)
    answer = raw_input("You sure? (y/n)")

    if answer == "y":
        
        for the_file in os.listdir(folder):
            file_path = os.path.join(folder, the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path): 
                    shutil.rmtree(file_path)

            except Exception as e:
                print(e)

        print("Success")
    else:
        print("Cancelled")
else:
    print("Empty directory")

Creating experiment folder: 54
Empty directory


In [392]:
experiment = vc.clone_experiment_to_storage(configName)
print(experiment)

INFO: [2018-10-01 09:59:25,956 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-10-01 09:59:25,992 - VirtualCoach] Experiment "fable_cerebellum" cloned successfully
fable_manipulation_dtu_spiking_cereb_v2.0_0


In [393]:
sim = vc.launch_experiment(experiment)

INFO: [2018-10-01 09:59:29,780 - VirtualCoach] Preparing to launch fable_manipulation_dtu_spiking_cereb_v2.0_0.
INFO: [2018-10-01 09:59:29,781 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-10-01 09:59:29,785 - VirtualCoach] Retrieving list of available servers.
[{u'gzweb': {u'assets': u'http://localhost:8080/assets',
             u'nrp-services': u'http://localhost:8080',
             u'videoStreaming': u'http://localhost:8080/webstream/',
             u'websocket': u'ws://localhost:8080/gzbridge'},
  u'id': u'localhost',
  u'rosbridge': {u'websocket': u'ws://localhost:8080/rosbridge'},
  u'serverJobLocation': u'local'}]
INFO: [2018-10-01 09:59:29,790 - Simulation] Attempting to launch fable_manipulation_dtu_spiking_cereb_v2.0_0 on localhost.
INFO: [2018-10-01 09:59:40,029 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #52)] Simulation Successfully Created.
INFO: [2018-10-01 09:59:40,033 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - lo

In [394]:
run_time = 200

sim.start()

walltime_start = time.time()

time_started = rospy.Time.now()
delta_t = rospy.Time.now() - time_started

print("\nStart delta_t: " + str(delta_t.to_sec()) + " (Should be 0)")

f = FloatProgress(min=0, max=run_time) # instantiate the bar

print("Simulating " + str(run_time) + " seconds. Simulation Progress:")
display(f) # display the bar

offset = rospy.Duration(0)
first_run = True
try:
    while delta_t.to_sec() < run_time:
        delta_t = rospy.Time.now() - time_started - offset
        
        if first_run:
            if delta_t.to_sec() < 0:
                offset = delta_t
            first_runt = False
            
        #print delta_t.to_sec(), "(Time offset: ",offset.to_sec(), ")"
        f.value = delta_t.to_sec()
        time.sleep(3)
except KeyboardInterrupt:
    pass

f.bar_style='danger'

sim.pause()
#sim.stop()
walltime_end = time.time()
print("\nSimulation paused at: " + str(delta_t.to_sec()) + ", executed in " + str(walltime_end - walltime_start) + " sec")

INFO: [2018-10-01 10:00:42,145 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #52)] Attempting to transition to state: started
INFO: [2018-10-01 10:00:42,151 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #52)] Simulation state: started

Start delta_t: 0.0 (Should be 0)
Simulating 200 seconds. Simulation Progress:


FloatProgress(value=0.0, max=200.0)

ERROR: [2018-10-01 10:00:42,437 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #52)] There was a Runtime error resulting from the Transfer Function. The full error is below:
 severity: 1
sourceType: "Transfer Function"
errorType: "Runtime"
message: "'NoneType' object has no attribute 'data'"
functionName: "controller"
lineNumber: 128
offset: -1
lineText: ''
fileName: ''
INFO: [2018-10-01 10:37:02,329 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #52)] Attempting to transition to state: paused
INFO: [2018-10-01 10:37:02,336 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #52)] Simulation state: paused

Simulation paused at: 200.008, executed in 2180.18542194 sec


In [395]:
sim.print_csv_file_names()
sim.save_csv()

###csv_data = {'planner_TF' : sim.get_csv_data('joint_info.csv'), 
###            'controller_TF': sim.get_csv_data('command.csv')}

csv_directory = "/home/dtu-neurorobotics/.opt/nrpStorage/" + experiment +"/"

file_list = os.listdir(csv_directory)
for i in file_list:
    if os.path.isdir(csv_directory+i):
        shutil.copyfile(csv_directory+i+"/"+"joint_info.csv", folder+"/"+"joint_info.csv")
        shutil.copyfile(csv_directory+i+"/"+"command.csv", folder+"/"+"command.csv")

###print "\nLaste entry in CSV at time :", csv_data['planner_TF'][-1][0]
###print ""

sim.stop()


INFO: [2018-10-01 10:37:40,228 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #52)] Attempting to retrieve recorders' csv data
[u'command.csv', u'joint_info.csv']
INFO: [2018-10-01 10:37:40,813 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #52)] Saved CSV data
INFO: [2018-10-01 10:37:40,821 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #52)] Attempting to transition to state: stopped
INFO: [2018-10-01 10:37:41,121 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #52)] Simulation has been stopped.
INFO: [2018-10-01 10:37:41,841 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #52)] Simulation state: stopped


In [396]:
sim.stop()
vc.delete_cloned_experiment(experiment)

INFO: [2018-10-01 10:37:48,591 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #52)] Attempting to transition to state: stopped
INFO: [2018-10-01 10:37:49,596 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #52)] Simulation state: stopped
INFO: [2018-10-01 10:37:49,597 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-10-01 10:37:49,606 - VirtualCoach] Experiment "fable_manipulation_dtu_spiking_cereb_v2.0_0" deleted successfully


In [397]:
vc.print_cloned_experiments()

INFO: [2018-10-01 10:37:52,220 - VirtualCoach] Retrieving list of experiments.
+------+
| Name |
+======+
+------+


In [334]:
sim.stop()

INFO: [2018-09-26 11:38:18,299 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #94)] Attempting to transition to state: stopped
ERROR: [2018-09-26 11:38:18,303 - bbp_client.oidc.client] Failed to retrieve data: Internal Server Error


HTTPError: HTTP Error 500: Invalid response 500.

In [123]:
sim.print_csv_file_names()
sim.save_csv()

###csv_data = {'planner_TF' : sim.get_csv_data('joint_info.csv'), 
###            'controller_TF': sim.get_csv_data('command.csv')}

csv_directory = "/home/dtu-neurorobotics/.opt/nrpStorage/" + experiment +"/"

file_list = os.listdir(csv_directory)
for i in file_list:
    if os.path.isdir(csv_directory+i):
        shutil.copyfile(csv_directory+i+"/"+"joint_info.csv", folder+"/"+"joint_info.csv")
        shutil.copyfile(csv_directory+i+"/"+"command.csv", folder+"/"+"command.csv")

###print "\nLaste entry in CSV at time :", csv_data['planner_TF'][-1][0]
###print ""

sim.stop()


INFO: [2018-09-18 12:41:29,073 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #13)] Attempting to retrieve recorders' csv data
[]
INFO: [2018-09-18 12:41:29,091 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #13)] Saved CSV data


IOError: [Errno 2] No such file or directory: '/home/dtu-neurorobotics/.opt/nrpStorage/fable_manipulation_dtu_spiking_cereb_v2.0_0/csv_records_2018-09-18_12-41-29/joint_info.csv'

In [126]:
vc.delete_cloned_experiment(experiment)

INFO: [2018-09-18 12:44:55,080 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-09-18 12:44:55,094 - VirtualCoach] Experiment "fable_manipulation_dtu_spiking_cereb_v2.0_0" deleted successfully


In [125]:
vc.print_cloned_experiments()

INFO: [2018-09-18 12:44:54,382 - VirtualCoach] Retrieving list of experiments.
+---------------------------------------------+
|                    Name                     |
+=============================================+
| fable_manipulation_dtu_spiking_cereb_v2.0_0 |
+---------------------------------------------+


In [322]:
vc.delete_cloned_experiment("fable_manipulation_dtu_spiking_cereb_v2.0_0")

INFO: [2018-09-26 11:36:49,467 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-09-26 11:36:49,495 - VirtualCoach] Experiment "fable_manipulation_dtu_spiking_cereb_v2.0_0" deleted successfully
